In [51]:
import openai, json
from IPython.core.magic_arguments import argument
from openai.types.chat import ChatCompletionMessage

client = openai.OpenAI()
messages = []

In [52]:
def get_weather(location):
    return "33 degrees celcius."


FUNCTION_MAP = {
    'get_weather': get_weather
}


In [53]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the weather for a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA"
                    }
                },
                "required": ["city"]
            }
        }
    }
]

In [56]:
from openai.types.chat import ChatCompletionMessage
def process_ai_response(msg: ChatCompletionMessage):
    print(msg)
    if msg.tool_calls:
        messages.append({
            "role": "assistant",
            "content": msg.content or "",
            "tool_calls": [{
                "id": tool_call.id,
                "type": "function",
                "function": {
                    "name": tool_call.function.name,
                    "arguments": tool_call.function.arguments,
                }

            } for tool_call in msg.tool_calls]
        })

        for tool_call in msg.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments
            print(f"Calling function: {function_name} with {arguments}")
            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                arguments = {}
            function_to_run = FUNCTION_MAP.get(function_name)
            result = function_to_run(**arguments)
            print(f"Ran: {function_name} with args: {arguments} for a result of {result}")
            messages.append({"role": "tool", "tool_call_id": tool_call.id, "name": function_name, "content": result})
            call_api()
    else:
        messages.append({"role": "assistant", "content": msg.content or ""})
        print(f"AI: {msg.content}")
    return msg.content


In [57]:
def call_api():
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=TOOLS
    )
    print(response)
    process_ai_response(response.choices[0].message)


In [60]:
while True:
    message = input("Send a message to the LLM..")
    if message == "quit":
        break
    else:
        messages.append({"role": "user", "content": message})
        print(f"User: {message}")
        call_api()

User: 난 동우야
ChatCompletion(id='chatcmpl-CICjYLZQncGPFjs8ci3sTobduhYZC', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='안녕하세요, 동우님! 또 뵙게 되어 반갑습니다. 어떻게 도와드릴까요?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1758454888, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=25, prompt_tokens=168, total_tokens=193, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
ChatCompletionMessage(content='안녕하세요, 동우님! 또 뵙게 되어 반갑습니다. 어떻게 도와드릴까요?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)
AI: 안녕하세요, 동우님! 또 뵙게 되어 반갑습니다. 어떻게 도와드릴까요?
User: 서울의 날씨가 어떄 ?
ChatCompleti

In [61]:
messages

[{'role': 'user', 'content': '나는 동우야'},
 {'role': 'assistant', 'content': '안녕하세요, 동우님! 어떤 도움이 필요하신가요?'},
 {'role': 'user', 'content': '파리의 날씨가 어때 ?'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'id': 'call_3BZOOx1ceJpoDMN7G63XuJKX',
    'type': 'function',
    'function': {'name': 'get_weather',
     'arguments': '{"location":"Paris, France"}'}}]},
 {'role': 'tool',
  'tool_call_id': 'call_3BZOOx1ceJpoDMN7G63XuJKX',
  'name': 'get_weather',
  'content': '33 degrees celcius.'},
 {'role': 'assistant',
  'content': '파리의 날씨는 현재 섭씨 33도입니다. 더운 날씨네요! 다른 궁금한 점이 있으신가요?'},
 {'role': 'user', 'content': '난 동우야'},
 {'role': 'assistant', 'content': '안녕하세요, 동우님! 또 뵙게 되어 반갑습니다. 어떻게 도와드릴까요?'},
 {'role': 'user', 'content': '서울의 날씨가 어떄 ?'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'id': 'call_rMgfjHDE1qijaMKmPFG5t5Lc',
    'type': 'function',
    'function': {'name': 'get_weather',
     'arguments': '{"location":"Seoul, South Korea"}'}}]},
 {'role': 'tool',
  'tool_call_id

In [50]:


a = '{"city": "San Francisco, CA"}'
b = json.loads(a)
print(b)
b["city"]


{'city': 'San Francisco, CA'}


'San Francisco, CA'